## Note: This Python script requires an Excel sheet "excel_input.xlsx" in the working directory to run

In [1]:
import numpy as np
import pandas as pd

In [2]:
from gurobipy import *
m = Model()

Academic license - for non-commercial use only


## Import Table Containing Cost Variable

In [3]:
table1 = pd.read_excel("excel_input.xlsx", sheet_name = 'table', index_col = 0)
table1

,vancouver,victoria,kelowna,calgary,edmonton,saskatoon,regina,winnipeg,toronto,ottawa,montreal,quebec,halifax
vancouver,99999,119,118,120,99,277,200,229,232,332,200,99999,99999
victoria,119,99999,99999,194,99999,99999,99999,99999,278,99999,99999,99999,99999
kelowna,118,99999,99999,151,102,99999,99999,99999,401,99999,99999,99999,99999
calgary,120,194,151,99999,147,169,182,129,159,278,231,99999,99999
edmonton,99,99999,102,147,99999,99999,99999,100,139,226,469,99999,99999
saskatoon,277,99999,99999,169,99999,99999,99999,99999,221,99999,99999,99999,99999
regina,200,99999,99999,182,99999,99999,99999,245,221,99999,99999,99999,99999
winnipeg,229,99999,99999,129,100,99999,245,99999,149,184,229,99999,99999
toronto,232,278,401,159,139,221,221,149,99999,122,115,147,166
ottawa,332,99999,99999,278,226,99999,99999,184,122,99999,380,99999,149


## Import Table Containing Constraints

In [4]:
constraint1 = pd.read_excel("excel_input.xlsx", sheet_name = 'constraint', header = None)

## Extract City Name

In [5]:
city_name = list(constraint1[0])

## Define a Function that Converts Variable Name to Corresponding City Name

In [6]:
def convert_num_to_city_name(number):
    if number < 14:
        city = city_name[number-1]
    else:
        remainder = number %14
        city = city_name[remainder-1]
    return city

## Create Dictionary of Constraints

In [7]:
rhs1 = {}
for i,val in constraint1.iterrows():
    rhs1['const' + str(i+1)] = int(constraint1[1][i])

## Define a Function that Execute Gurobi Optimization

In [8]:
obj_list = list()
dest_list = list()
variable_list = list()
complete_price_list = list()
complete_city_list = list()

In [9]:
def find_flight(table, rhs):
    ## This function takes a data frame and (flow out - flow in constraint) and return
    ## optimized result
    
    ## table = Data Frame Containing Cost Variable
    ## rhs = right hand side constraint

    m = Model()
    cost_matrix = {}
    for i in range(len(table)):
        cost_matrix[str(i+1)] = table.iloc[i,:].tolist()
    
    #Define iterable variable 
    num_of_origin = len(table)
    num_of_dest = len(table)
    
    ##Add Variable to Model
    decision_level = m.addVars(num_of_origin, num_of_dest)
    
    
    for i in range(0, num_of_origin):
        for j in range(0, num_of_dest):
            decision_level[i,j].vtype = GRB.CONTINUOUS
    
    ##Add constraint to model (flow out - flow in)
    m.addConstrs((quicksum((decision_level[i, j] ) for j in range(0, num_of_origin)) 
              - (quicksum((decision_level[j, i] ) for j in range(0, num_of_origin)))  == rhs['const' + str(i+1)] for i in range(0, num_of_dest)), 
             name= "flow")
    
    m.update() 
    
    ##Setup Objective Function
    obj = quicksum(quicksum(cost_matrix[str(i+1)][j] * decision_level[i, j] for i in range(0, num_of_origin)) for j in range(0, num_of_dest))
    
    ##Tell Gurobi to Minimize Objective Fucntion
    m.setObjective(obj, GRB.MINIMIZE)
    
    ##Solve the LP and look for optimal objective function value
    m.optimize()
    
    ##Extract Optimal Decision
    obj_list.append(m.objVal)
    
    ##Extract Optimal Decision Variable
    variable_list.append(m.x)
    
    ##Store Cost of Optimal Decision into a List
    temp_list = list()
    for i in range(num_of_origin):
        for j in range(num_of_dest):
            if decision_level[i,j].x != 0:
                x = cost_matrix[str(i+1)][j]
                temp_list.append(x)
    complete_price_list.append(temp_list)

## Call find_flight Function and Execute Gurobi Optimization

In [10]:
for z in range(2,14):
    
    ##Set a destination's constraint to ==> flow out - flow in = -1 
    rhs1['const' + str(z)] = -1
    
    ##Calls Optimization Function
    find_flight(table1, rhs1)
    
    ##Track destination and store it in a list
    dest_list.append(city_name[z-1])
    
    ##Reset Gurobi model for next iteration use
    m.reset()
    
    ##Reset right hand side constraint to ==> flow out - flow in = 0 for all destination
    rhs1['const' + str(z)] = 0

Optimize a model with 13 rows, 169 columns and 312 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 13 columns
Presolve time: 0.01s
Presolved: 13 rows, 156 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.000000e+00   0.000000e+00      0s
       2    1.1900000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  1.190000000e+02
Optimize a model with 13 rows, 169 columns and 312 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 0 rows and 13 columns
Presolve time: 0.01s
Presolved: 13 rows, 156 columns, 312 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.

In [11]:
lst1 = list()
for i, val in enumerate(variable_list):
    lst2 = list()
    for z, valz in enumerate(val):
        if valz == 1:
            lst2.append(z+1)
    lst1.append(lst2)

## Optimized Result

In [12]:
for i in range(len(obj_list)):
    
    if len(complete_price_list[i]) == 1 :
        print("Flying from Vancouver to {} \t: ${} \t (DIRECT FLIGHT)".format(dest_list[i], complete_price_list[i][0]))
    
    else:
        transit_city = convert_num_to_city_name(lst1[i][0])
        print("Flying from Vancouver to {} \t: ${} \t (TRANSIT)".format(dest_list[i], complete_price_list[i][0]+complete_price_list[i][1]))
        print("\t Vancouver to {} \t: ${}".format(transit_city, complete_price_list[i][0]))
        print("\t {} to {} \t: ${}".format(transit_city, dest_list[i], complete_price_list[i][1]))

Flying from Vancouver to victoria 	: $119 	 (DIRECT FLIGHT)
Flying from Vancouver to kelowna 	: $118 	 (DIRECT FLIGHT)
Flying from Vancouver to calgary 	: $120 	 (DIRECT FLIGHT)
Flying from Vancouver to edmonton 	: $99 	 (DIRECT FLIGHT)
Flying from Vancouver to saskatoon 	: $277 	 (DIRECT FLIGHT)
Flying from Vancouver to regina 	: $200 	 (DIRECT FLIGHT)
Flying from Vancouver to winnipeg 	: $199 	 (TRANSIT)
	 Vancouver to edmonton 	: $99
	 edmonton to winnipeg 	: $100
Flying from Vancouver to toronto 	: $232 	 (DIRECT FLIGHT)
Flying from Vancouver to ottawa 	: $325 	 (TRANSIT)
	 Vancouver to edmonton 	: $99
	 edmonton to ottawa 	: $226
Flying from Vancouver to montreal 	: $200 	 (DIRECT FLIGHT)
Flying from Vancouver to quebec 	: $379 	 (TRANSIT)
	 Vancouver to toronto 	: $232
	 toronto to quebec 	: $147
Flying from Vancouver to halifax 	: $398 	 (TRANSIT)
	 Vancouver to toronto 	: $232
	 toronto to halifax 	: $166
